In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.image import imread
import seaborn as sns
import cv2
import PIL
import pillow_avif
import tensorflow as tf
from sklearn.metrics import classification_report
import time
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization
from functools import partial

from sklearn.metrics import accuracy_score,precision_score,recall_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
data_dir = 'Landmarks_v0'

In [9]:
categories=[]
landmarks=[]
for i in os.listdir(data_dir):
    if i!='.DS_Store':
        categories.append(i)
for i in categories:
    for landmark in os.listdir(os.path.join(data_dir,i)):
        landmarks.append(landmark)

In [11]:
image_data = []
landmark_labels = []
category_labels = []
landmark_names = []
c=0
for category_idx, category in enumerate(categories):
    category_dir = os.path.join(data_dir, category)
    landmarks = os.listdir(category_dir)
    for landmark_idx, landmark in enumerate(landmarks):
        landmark_dir = os.path.join(category_dir, landmark)
        if landmark!='.DS_Store':
            for img in os.listdir(landmark_dir):
                img_path = os.path.join(landmark_dir, img)
                img_array = load_img(img_path, target_size=(224, 224))
                img_array = img_to_array(img_array)
                image_data.append(img_array)
                landmark_labels.append(c)

                category_labels.append(category_idx)
                landmark_names.append(landmark)
            c+=1

In [12]:
for category_idx, category in enumerate(categories):
    print(category,category_idx)

Pagodas 0
NeoClassical 1
Pyramids 2
Modern 3
Mughal 4
Gothic 5


In [13]:
landmark_list=[]
for i in range(len(landmark_labels)):
    landmark_list.append((landmark_names[i],landmark_labels[i]))

set_land = set(landmark_list)
land_list=[0 for i in range(30)]
for i in set_land:
    land_list[i[1]]=i[0]
print(land_list)

['TianningTemplePagoda', 'ThienMuPagoda', 'GiantWildGoosePagoda', 'ShwedagonPagoda', 'FogongTemplePagoda', 'Buckingham Palace', 'Concertgebouw', 'Panth‚on', 'Academy of Athens', 'Ripon Building', 'Pyramid of Giza', 'Pyramid of Djoser', 'Santa Cecilia Acatitlan Pyramid', 'El Castillo, Chichen Itza', 'Louvre Pyramid', 'Hallgr¡mskirkja', 'Chrysler Building', 'CCTV Headquarters', 'eiffel', 'Cathedral of Bras”lia', 'Tomb of I_timad-ud-Daulah', 'Taj Mahal', 'Tomb of Akbar', 'Jama Masjid', 'Bibi Ka Maqbara', 'St.VitusCathedral', 'MilanCathedral', 'ChartresCathedral', 'CologneCathedral', 'Notre-DameCathedral']


In [15]:
# Split the data into train and test sets
X_train, X_test, y_landmark_train, y_landmark_test, y_category_train, y_category_test, landmarks_train, landmarks_test = train_test_split(
    image_data, landmark_labels, category_labels, landmark_names, test_size=0.2, random_state=42, stratify=landmark_labels
)

num_category_classes = 6
num_landmark_classes = 30

y_category_train = to_categorical(y_category_train, num_classes=num_category_classes)
y_category_test = to_categorical(y_category_test, num_classes=num_category_classes)

y_landmark_train = to_categorical(y_landmark_train, num_classes=num_landmark_classes)
y_landmark_test = to_categorical(y_landmark_test, num_classes=num_landmark_classes)

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#from tensorflow.keras.applications.vgg_preprocess import preprocess_input as vgg_preprocess

BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
image_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
                               rotation_range=20, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1, 
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [17]:
train_data_gen_category = image_gen.flow(
        np.array(X_train),
        np.array(y_category_train)
    )

test_data_gen_category = image_gen.flow(
    np.array(X_test),
    np.array(y_category_test)
)

# Create data generators for train and test sets for landmarks
train_data_gen_landmark = image_gen.flow(
    np.array(X_train),
    np.array(y_landmark_train)
)

test_data_gen_landmark = image_gen.flow(
    np.array(X_test),
    np.array(y_landmark_test),
    shuffle=False
)

In [18]:
import keras_tuner as kt

def build_model(hp):
    vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    num_layers_to_unfreeze = hp.Int("num_layers_to_unfreeze", min_value=1, max_value=5, step=1)
    for layer in vgg16_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False
    
    x = vgg16_model.output
    x = Flatten()(x)
    x = Dense(hp.Int("dense_1_units", min_value=512, max_value=2048, step=256), activation='relu')(x)
    x = Dropout(hp.Float("dropout_1_rate", min_value=0.3, max_value=0.7, step=0.1))(x)
#     x = BatchNormalization()(x)
    x = Dense(hp.Int("dense_2_units", min_value=512, max_value=2048, step=256), activation='relu')(x)
    x = Dropout(hp.Float("dropout_2_rate", min_value=0.3, max_value=0.7, step=0.1))(x)
    x = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=vgg16_model.input, outputs=x)
    
    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="LOG")),
        metrics=["accuracy"],
    )
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter configurations to try
    executions_per_trial=1,  # Number of trainings per trial
    directory='category_tuning',
    project_name='category_model_tuning'
)

INFO:tensorflow:Reloading Tuner from category_tuning/category_model_tuning/tuner0.json


In [19]:
from tensorflow.keras.callbacks import EarlyStopping

X_train_category = train_data_gen_category.x
y_train_category = train_data_gen_category.y
X_test_category = test_data_gen_category.x
y_test_category = test_data_gen_category.y

tuner.search_space_summary()

early_stop = EarlyStopping(monitor="val_accuracy", patience=2, verbose=1)
tuner.search(train_data_gen_category, validation_data=test_data_gen_category, epochs=5, callbacks=[early_stop])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best dense_1_units: {best_hps.get('dense_1_units')}")
print(f"Best dropout_1_rate: {best_hps.get('dropout_1_rate')}")
print(f"Best dense_2_units: {best_hps.get('dense_2_units')}")
print(f"Best dropout_2_rate: {best_hps.get('dropout_2_rate')}")
print(f"Best learning_rate: {best_hps.get('learning_rate')}")

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_category,y_train_category, validation_data=(X_test_category,y_test_category), epochs=10)


Trial 11 Complete [00h 03m 18s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.9047619104385376
Total elapsed time: 00h 03m 18s
INFO:tensorflow:Oracle triggered exit
Best dense_1_units: 768
Best dropout_1_rate: 0.4
Best dense_2_units: 2048
Best dropout_2_rate: 0.5
Best learning_rate: 0.0002995853178427231
Epoch 1/10
11/11 [==============================] - 37s 3s/step - loss: 13.8047 - accuracy: 0.4077 - val_loss: 0.6351 - val_accuracy: 0.9048
Epoch 2/10
11/11 [==============================] - 37s 3s/step - loss: 1.8967 - accuracy: 0.8155 - val_loss: 0.5585 - val_accuracy: 0.9048
Epoch 3/10
11/11 [==============================] - 37s 3s/step - loss: 1.3356 - accuracy: 0.9137 - val_loss: 0.7620 - val_accuracy: 0.9048
Epoch 4/10
11/11 [==============================] - 37s 3s/step - loss: 0.5259 - accuracy: 0.9583 - val_loss: 0.3196 - val_accuracy: 0.9405
Epoch 5/10
11/11 [==============================] - 37s 3s/step - loss: 0.3562 - accuracy: 0.9643 - val_loss: 0.4968 

In [11]:
model.evaluate(X_test_category, y_test_category)

3/3 [==============================] - 7s 2s/step - loss: 0.5415 - accuracy: 0.9286


[0.541456937789917, 0.9285714030265808]

In [21]:
X_train_landmark = train_data_gen_landmark.x
y_train_landmark = train_data_gen_landmark.y
X_test_landmark = test_data_gen_landmark.x
y_test_landmark = test_data_gen_landmark.y

# Prepare the category predictions for the train dataset
category_predictions_train = model.predict(X_train_landmark)

# Prepare the category predictions for the test dataset
category_predictions_test = model.predict(X_test_landmark)

3/3 [==============================] - 7s 2s/step


In [25]:
def build_landmark_model(hp):
    # Load the base VGG16 model for landmark classification
    vgg16_landmark_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    num_layers_to_unfreeze = hp.Int("num_layers_to_unfreeze_landmark", min_value=1, max_value=5, step=1)
    for layer in vgg16_landmark_base.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False
#     for layer in vgg16_landmark_base.layers[:-3]:
#         layer.trainable = False
    
    # Extract the output of the base VGG16 model for landmark classification
    vgg_landmark_output = vgg16_landmark_base.output

    # Flatten the VGG16 landmark output
    feature_vector_landmark = Flatten(name='flatten_landmark')(vgg_landmark_output)

    # Use the category predictions as input
    category_input_landmark = Input(shape=(6,), name='category_input_landmark')

    # Concatenate the feature vector and the category input
    merged_input_landmark = Concatenate(name="concatenate_landmark")([feature_vector_landmark, category_input_landmark])

    # Tune the number of units in the first Dense layer
    units = hp.Int('units', min_value=512, max_value=2048, step=256)
    x_landmark = Dense(units, activation='relu')(merged_input_landmark)
    
    # Tune the dropout rate
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    x_landmark = Dropout(dropout_rate)(x_landmark)
    
    x_landmark = BatchNormalization()(x_landmark)
    
    # Tune the number of units in the first Dense layer
    units = hp.Int('units2', min_value=512, max_value=2048, step=256)
    x_landmark = Dense(units, activation='relu')(x_landmark)
    
    # Tune the dropout rate
    dropout_rate = hp.Float('dropout_rate2', min_value=0.1, max_value=0.5, step=0.1)
    x_landmark = Dropout(dropout_rate)(x_landmark)

    landmark_output = Dense(30, activation='softmax')(x_landmark)

    # Create the landmark model
    landmark_model = Model(inputs=[vgg16_landmark_base.input, category_input_landmark], outputs=landmark_output)

    landmark_model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

    return landmark_model


In [26]:
tuner = RandomSearch(
    build_landmark_model,
    objective='val_accuracy',
    max_trials=15,  # Number of different hyperparameter configurations to try
    executions_per_trial=1,  # Number of trainings per trial
    directory='monday_mrng_tuner',
    project_name='landmark_model_tuning'
)

tuner.search_space_summary()

# Start the search for the best hyperparameters
tuner.search(
    [X_train_landmark, category_predictions_train],
    y_train_landmark,
    epochs=10,
    validation_data=([X_test_landmark, category_predictions_test], y_test_landmark)
)

INFO:tensorflow:Reloading Tuner from monday_mrng_tuner/landmark_model_tuning/tuner0.json
Search space summary
Default search space size: 5
num_layers_to_unfreeze_landmark (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 2048, 'step': 256, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 2048, 'step': 256, 'sampling': 'linear'}
dropout_rate2 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
INFO:tensorflow:Oracle triggered exit


In [27]:
# Get the best model
best_landmark_model = tuner.get_best_models(num_models=1)[0]
best_landmark_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [28]:
# Evaluate the best model
results = best_landmark_model.evaluate([X_test_landmark, category_predictions_test], y_test_landmark)
print("Test loss: ", results[0])
print("Test accuracy: ", results[1])

3/3 [==============================] - 7s 2s/step - loss: 0.4183 - accuracy: 0.9048
Test loss:  0.4182627499103546
Test accuracy:  0.9047619104385376


In [20]:
model.save('category_model_best.h5')

In [80]:
best_landmark_model.save('landmark_model_best.h5')